# On Classifier Evaluation

In [1]:
from sklearn.metrics import confusion_matrix
import numpy as np
from sklearn.utils import resample
from sklearn.datasets import load_breast_cancer
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LogisticRegression

## Confusion Matrix

For classification problems, the target is a categorical variable. This means that we can simply count the number of times that our model predicts the correct category and the number of times that it predicts something else.

We can visualize this by means of a **confusion matrix**, which displays counts of correct and incorrect predictions. We'll explore this below. There are [many ways](https://docs.google.com/document/d/12BkMOXt5-iMzw_y_zFLZNkPlgAy1rj1t9gJ6LXi2bqQ/edit?usp=sharing) of evaluating a classification model, but most derive from the confusion matrix.

## Lottery Number Prediction

Suppose I want to train a model to predict whether a string of six numbers (a "ticket") would win the lottery or not. What sort of data might I use?

### Scenario 1

I gather all the winning tickets from the last 10000 days or so. So I have one column for the tickets themselves, and a Boolean column indicating whether the ticket was a winner or not.

Now if all the tickets on which my model trains are *winning* tickets, then it would predict every ticket to win! Suppose I test the model on a set of 1000 tickets, and suppose that there is exactly one winning ticket among those 1000. My model will always predict the ticket to win. Let's think about what the confusion matrix will look like.

In [2]:
# Setting up the true values

y_true = np.zeros(1000)
y_true[500] = 1

y_true

array([0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0.

In [3]:
# Setting up the predictions

y_pred = np.ones(1000)

y_pred

array([1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
       1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
       1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
       1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
       1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
       1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
       1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
       1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
       1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
       1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
       1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
       1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
       1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
       1., 1., 1., 1., 1.

In [4]:
# Defining the confusion matrix

cm_1 = confusion_matrix(y_true, y_pred)

The confusion matrix should tell us that we have 999 false positives (999 losing tickets predicted to win) and 1 true positive (1 winning ticket predicted to win):

In [5]:
cm_1

array([[  0, 999],
       [  0,   1]])

Notice the way that sklearn displays its confusion matrix: The rows are \['actually false', 'actually true'\]; the columns are \['predicted false', 'predicted true'\].

So it displays:

$\begin{bmatrix}
TN & FP \\
FN & TP
\end{bmatrix}$

In [6]:
tn = cm_1[0, 0]
fp = cm_1[0, 1]
fn = cm_1[1, 0]
tp = cm_1[1, 1]

Let's see if we can calculate some of our metrics for this matrix.

**Accuracy** = $\frac{TP + TN}{TP + TN + FP + FN}$

In words: How often did my model get the right answer?

In [7]:
# Accuracy

print(f'Accuracy: ({tp} + {tn}) / ({tp} + {tn} + {fp} + {fn}) = 1/1000 = {1/1000}')

Accuracy: (1 + 0) / (1 + 0 + 999 + 0) = 1/1000 = 0.001


**Recall** = $\frac{TP}{TP + FN}$

In words: How often did my model correctly predict winning tickets?

In [8]:
# Recall

print(f'Recall: {tp} / ({tp} + {fn}) = 1/1 = {1}')

Recall: 1 / (1 + 0) = 1/1 = 1


**Precision** = $\frac{TP}{TP + FP}$

In words: How often was my model's prediction of 'winner' correct?

In [9]:
# Precision

print(f'Precision: {tp} / ({tp} + {fp}) = 1 / 1000 = {1/1000}')

Precision: 1 / (1 + 999) = 1 / 1000 = 0.001


**F-1 Score** = $\frac{2PrRc}{Pr + Rc}$ = $\frac{2TP}{2TP + FP + FN}$

In [10]:
# F-1 Score

print(f'F-1 Score: (2)({tp}) / ((2)({tp}) + {fp} + {fn}) = 2 / 1001 = {round(2/1001, 3)}')

F-1 Score: (2)(1) / ((2)(1) + 999 + 0) = 2 / 1001 = 0.002


### Scenario 2

Well, my recall was good, but everything else I measured was terrible! Can I do better?

This time I'll train my model in a much different way. I'll give it the tickets of 10000 people who played the lottery yesterday. Suppose that there is one winning ticket and 9999 losers. Now I test the model on the same 1000 tickets from before in Scenario 1.

This time my model will almost always predict a ticket to lose. Suppose that, in the 1000 predictions, it makes only one prediction of a winner, and suppose that this prediction is wrong.

#### Exercise: Set up the confusion matrix and compute our metrics for this new scenario.

In [11]:
y_true2 = np.zeros(1000)
y_true2[0] = 1

y_pred2 = np.zeros(1000)
y_pred2[1] = 1

In [14]:
cm_2 = confusion_matrix(y_true2, y_pred2)
cm_2

array([[998,   1],
       [  1,   0]])

In [16]:
tn2 = cm_2[0, 0]
fp2 = cm_2[0, 1]
fn2 = cm_2[1, 0]
tp2 = cm_2[1, 1]

print('accuracy:' + str((tp2 + tn2) / (tn2 + fp2 + fn2 + tp2)))
print('recall:' + str(tp2 / (tp2 + fn2)))
print('precision:' + str(tp2 / (tp2 + fp2)))
print('f-1 score:' + str(2*tp2 / (2*tp2 + fp2 + fn2)))

accuracy:0.998
recall:0.0
precision:0.0
f-1 score:0.0


### Lessons

The last classifier had a really high accuracy, but everything else was terrible.

It occurs to me now that if I'm really going to get a good model running, I'm going to have to have good numbers of **both winning and losing  tickets**. The first model didn't see enough losers, and the second model didn't see enough winners.

Of course, I could just be more careful about my data collection, but there are often easier fixes. One of the most effective strategies is **oversampling the minority class**. That is, I give myself more data points than I really have. I could achieve this either by [bootstrapping](https://scikit-learn.org/stable/modules/generated/sklearn.utils.resample.html) or by generating some data that is fake but close to actual data. The latter is the idea behind [SMOTE](https://imbalanced-learn.readthedocs.io/en/stable/generated/imblearn.over_sampling.SMOTE.html).

## Breast Cancer Prediction

Let's see what results we get on scikit-learn's breast cancer dataset:

### Loading the Data

In [17]:
preds, target = load_breast_cancer(return_X_y=True)

One way to measure the relative size of our classes:

In [18]:
sum(target) / len(target)

0.6274165202108963

Let's assume that this is close enough to "balanced"!

### Splitting and Scaling:

In [19]:
X_train, X_test, y_train, y_test = train_test_split(preds, target)

In [20]:
ss = StandardScaler()

ss.fit(X_train)

X_train_sc = ss.transform(X_train)
X_test_sc = ss.transform(X_test)

### Model Fitting

In [21]:
logreg = LogisticRegression(solver='lbfgs', max_iter=10000)

logreg.fit(X_train_sc, y_train)

LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
                   intercept_scaling=1, l1_ratio=None, max_iter=10000,
                   multi_class='auto', n_jobs=None, penalty='l2',
                   random_state=None, solver='lbfgs', tol=0.0001, verbose=0,
                   warm_start=False)

### Confusion Matrix

In [22]:
confusion_matrix(y_test, logreg.predict(X_test_sc))

array([[53,  3],
       [ 2, 85]])

### Interpretation

#### Exercise: What are the precision, recall, and accuracy of our model?

In [23]:
precision = 85 / (85 + 3)
precision

0.9659090909090909

In [24]:
recall = 85 / (85 + 2)
recall

0.9770114942528736

In [25]:
accuracy = (53 + 85) / (53 + 3 + 2 + 85)
accuracy

0.965034965034965